In [20]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import string
from string import punctuation
from nltk.corpus import stopwords

stop_words = set(stopwords.words('russian'))
punctuation = punctuation + '\n' + '—' + '“' + ',' + '”' + '‘' + '-' + '’'

In [34]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    
def preprocessing(article):
    global article_sent
    article = article.str.lower()
    article = article.apply(lambda x: cleanhtml(x))
    article = article.apply(lambda x: re.sub('\S+@\S+','', x))
    article = article.apply(lambda x: re.sub("((http\://|https\://|ftp\://)|(www.))+(([a-zA-Z0-9\.-]+\.[a-zA-Z]{2,4})|([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}))(/[a-zA-Z0-9%:/-_\?\.'~]*)?",'', x))
    article = article.apply(lambda x: x.replace("\xa0", " "))
    article = article.apply(lambda x: x.replace("'s", ''))
    article = article.apply(lambda x: x.replace('’s', ''))
    article = article.apply(lambda x: x.replace("\'s", ''))
    article = article.apply(lambda x: x.replace("\’s", ''))
    article = article.apply(lambda x: re.sub(' +', ' ',x))
    article_sent = article.copy()
    article = article.apply(lambda x: ''.join(word for word in x if word not in punctuation))
    article = article.apply(lambda x: re.sub(' +', ' ',x))
    article = article.apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
    
    return article

def normalize(li_word):
    global normalized_freq
    normalized_freq = []
    for dictionary in li_word:
        max_frequency = max(dictionary.values())
        for word in dictionary.keys():
            dictionary[word] = dictionary[word]/max_frequency
        normalized_freq.append(dictionary)
        
    return normalized_freq

def word_frequency(article_word):
    word_frequency = {}
    li_word = []
    for sentence in article_word:
        for word in word_tokenize(sentence):
            if word not in word_frequency.keys():
                word_frequency[word] = 1
            else:
                word_frequency[word] += 1
        li_word.append(word_frequency)
        word_frequency = {}
    normalize(li_word)
    
    return normalized_freq

def sentence_score(li):
    global sentence_score_list
    sentence_score = {}
    sentence_score_list = []
    for list_, dictionary in zip(li, normalized_freq):
        for sent in list_:
            for word in word_tokenize(sent):
                if word in dictionary.keys():
                    if sent not in sentence_score.keys():
                        sentence_score[sent] = dictionary[word]
                    else:
                        sentence_score[sent] += dictionary[word]
        sentence_score_list.append(sentence_score)
        sentence_score = {}
        
    return sentence_score_list

def sent_token(article_sent):
    sentence_list = []
    sent_token = []
    for sent in article_sent:
        token = sent_tokenize(sent)
        for sentence in token:
            token_2 = ''.join(word for word in sentence if word not in punctuation)
            token_2 = re.sub(' +', ' ',token_2)
            sent_token.append(token_2)
        sentence_list.append(sent_token)
        sent_token = []
    sentence_score(sentence_list)
    
    return sentence_score_list

def summary(sentence_score):
    summary_list = []
    for summ in sentence_score:
        select_length = int(len(summ)*0.25)
        summary_ = nlargest(select_length, summ, key = summ.get)
        summary_list.append(".".join(summary_))
        
    return summary_list

def to_series(art):
    global dataframe
    data_dict = {'article' : [art]}
    dataframe = pd.DataFrame(data_dict)['article']
    
    return dataframe

def article_summarize(artefact):
    
    if type(artefact) != pd.Series:
        artefact = to_series(artefact)
    df = preprocessing(artefact)
    word_normalization = word_frequency(df)
    sentence_score = sent_token(article_sent)
    summarized_article = summary(sentence_score)
    return summarized_article

In [35]:
summaries = article_summarize(article)
print(summaries)

['в смысловой цельности текста отражаются те связи и зависимости которые имеются в самой действительности общественные события явления природы человек его внешний облик и внутренний мир предметы неживой природы и т']


In [32]:
from summa.summarizer import summarize
#библиотечный TextRank, для сравнения
def predict_text_rank(text, summary, summary_part=0.1):
    return summarize(text, ratio=summary_part, language='russian').replace("\n", " ")

#calc_method_score(test_obj, predict_text_rank)